In [ ]:
from websockets.sync.client import connect
import json

In this example we use the websockets library (https://websockets.readthedocs.io/en/stable/)

This is a super simple way to receive the websocket data, but feel free to use other methods.

In [ ]:
def receive_data():
    max_msg = 10 # Feel free to change this or just let it loop forever. Maybe best to keep it low while testing...
    with connect("ws://192.87.172.71:1337") as websocket:
        count = 0
        while count < max_msg:
            msg = websocket.recv()
            try:
                msg = json.loads(msg)
                handle_message(msg) # Call function to do the actual data handling
            except json.JSONDecodeError:
                print("Failed to decode json, assuming next packet will be ok...")
                pass
            except Exception as e:
                # Assume something went wrong and stop receiving
                print("Something went horribly wrong!")
                print("Error:", e)
                break
            
            count += 1
        print("Received %d messages!" % count)

# 'Packet' format
The websocket will give you one entry per packet that was received on a gateway. This packet can have the following fields:

- `tmst`: internal timestamp of the receiver, you can ignore this as we have a UTC timetamp
- `time`: Timestamp the gateway received the packet
- `chan`: lora channel
- `rfch`: 'rf chain' used
- `freq`: frequency the packet was received on
- `stat`: CRC status (if the packet passed the error correction check). I'm, 99% sure we only get 'good' packets so this should always be 1.
- `modu`: Moduleation, 'LORA' or 'FSK' (But I think we only get LORA)
- `datr`: Spreading factor and bandwidth as a string [https://www.thethingsnetwork.org/docs/lorawan/spreading-factors/]
- `codr`: Coding rate [https://www.thethingsnetwork.org/docs/lorawan/fec-and-code-rate/]
- `lsnr`: Lora SNR ratio in db [https://www.thethingsnetwork.org/docs/lorawan/rssi-and-snr/]
- `rssi`: Signal strength indication at the receiver (gateway) in dBm
- `size`: size of the packet in bytes(?)
- `gateway`: Address of the gateway, you can compare this to the list of gateway locations.
- `device_addr`: The 'short' device address. These might change!
- `device_eui`: The actual device address. Note that this will only be included if we know the device!
- `device_name`: The device name. Note that this will only be included if we know the device!

Note: the same message may be received by multiple gateways, so don't be surprised if you see the same message multiple times. In fact, this is one of the properties you could use to learn more about signal propagation and signal quality.

In [ ]:
def handle_message(msg):
    print("Received packet with rssi: %d." % msg['rssi'])
    print(msg)
    if 'device_eui' in msg:
        print("This is a known device!")
        print("Device name: ", msg['device_name'])
    print()

In [ ]:
receive_data()